In [1]:
import numpy as np
import os
import csv
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from deepgaze.color_detection import RangeColorDetector
from deepgaze.mask_analysis import BinaryMaskAnalyser

C:\Users\Rou\AppData\Local\Temp\ipykernel_16052\747322137.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# 假设my_skin_detector已经定义并包含returnMask方法
class MySkinDetector:
    def returnMask(self, image, morph_opening=True, blur=True, kernel_size=3, iterations=1):
        # 这里是一个示例实现，你需要根据你的实际情况进行修改
        # 转换到HSV色彩空间
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        lower = np.array([0, 48, 80], dtype="uint8")
        upper = np.array([20, 255, 255], dtype="uint8")
        mask = cv2.inRange(hsv, lower, upper)
        if morph_opening:
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=iterations)
        if blur:
            mask = cv2.GaussianBlur(mask, (3, 3), 0)
        return mask

# 实例化MySkinDetector
my_skin_detector = MySkinDetector()


In [3]:
# 扫描目录并解析文件路径，收集所有脸部位置
def scan_directory_and_parse(directory):
    file_data = []
    face_positions = set()
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                file_path = os.path.join(root, file)
                
                # 提取目录中的ID、日期和时间
                base_directory = os.path.basename(root)
                ID, date, time = base_directory.split('_')
                
                # 转换日期和时间格式
                date = f"{date[:4]}/{date[4:6]}/{date[6:]}"
                time = f"{time[:2]}:{time[2:]}"
                
                # 提取文件名中的脸部位置和Frame编号
                position, frame_number = file.split('_')
                frame_number = frame_number.split('.')[0]  # 去掉文件扩展名
                
                # 存储提取的信息
                file_data.append([ID, f"{date} {time}", position, frame_number, file_path])
                face_positions.add(position)
    
    return file_data, sorted(face_positions)


In [4]:
# 处理图像并记录膚色像素变化值
def process_images_and_record_changes(file_data, face_positions):
    skin_pixels_changes = {}
    previous_skin_pixels = {}

    for data in file_data:
        ID, datetime, position, frame_number, file_path = data
        
        # 初始化字典
        if ID not in skin_pixels_changes:
            skin_pixels_changes[ID] = {position: [] for position in face_positions}
            previous_skin_pixels[ID] = {position: 0 for position in face_positions}
        
        # 读取图像
        image = cv2.imread(file_path)

        # 生成膚色遮罩
        image_mask = my_skin_detector.returnMask(image, morph_opening=True, blur=True, kernel_size=3, iterations=1)

        # 统计膚色像素数量
        skin_pixels = np.count_nonzero(image_mask)

        # 计算膚色像素数量的变化
        skin_pixels_change = skin_pixels - previous_skin_pixels[ID][position]

        # 更新前一影像的膚色像素数量
        previous_skin_pixels[ID][position] = skin_pixels

        # 将膚色變化值添加到字典中
        skin_pixels_changes[ID][position].append((datetime, frame_number, skin_pixels_change))

    return skin_pixels_changes

In [5]:
# 将结果写入CSV文件
def write_to_csv(file_data, face_positions, skin_pixels_changes, csv_file_path, app_data, app_header):
    with open(csv_file_path, mode='w', newline='', encoding="utf-8-sig") as file:
        writer = csv.writer(file)
        header = app_header + ['Frame'] + face_positions
        writer.writerow(header)
        
        # 写入数据
        for data in file_data:
            ID, datetime, position, frame_number, file_path = data
            key = (ID, datetime)
            if key in app_data:
                app_row = app_data[key]
                row = app_row + [frame_number]
                row += [next((change for dt, fn, change in skin_pixels_changes[ID][position] if fn == frame_number), '') for position in face_positions]
                writer.writerow(row)

In [6]:
def read_uLifePlusCSV(app_csv_path):
    # 读取uLifePlusApp.csv文件
    app_data = {}
    app_header =[]
    with open(app_csv_path, mode='r', newline='', encoding="utf-8") as file:
        app_df = pd.read_csv(file, dtype={'編號': str})  # 确保编号保持原来的格式
        app_header = app_df.columns.tolist()  # 获取标题行
        for _, row in app_df.iterrows():
            key = (row['編號'], row['量測日期時間'])
            app_data[key] = row.tolist()  # 存储整行数据
    return app_data,app_header


In [7]:
directory = 'pngFromVideo'
app_csv_path = 'ulife/uLifePlusApp_20240531_20240607.csv'
app_data, app_header= read_uLifePlusCSV(app_csv_path)
file_data, face_positions = scan_directory_and_parse(directory)
skin_pixels_changes = process_images_and_record_changes(file_data, face_positions)

# 写入CSV文件
csv_file_path = 'HSV_output/skin_pixel_changes.csv'
print(app_data)
write_to_csv(file_data, face_positions, skin_pixels_changes, csv_file_path, app_data, app_header)
print(f"File information has been saved to {csv_file_path}")



{('022', '2024/06/07 11:11'): ['022', 'ＹＣ', '男性', '1985/08/12', 38, '優於同年齡', '2024/06/07 11:11', '坐姿', 80, 58.6, 56.0, 2.07, 3990.2, 304.9, 2637.9, 10.45, 69.0, 1.02, 1047.4, 8.77, 2.5, 0.72, '數值偏高', '輕度鬆散', '輕度', '正常範圍', '極重度', '數值偏高', '正常範圍', '正常範圍', '極重度', '正常範圍', '重度', '數值偏高，疑似心律不整，追蹤觀察！', '【注意】'], ('025', '2024/06/07 10:05'): ['025', 'Nick', '男性', '1987/07/08', 36, '優於同年齡', '2024/06/07 10:05', '坐姿', 81, 22.0, 34.4, 1.24, 723.6, 182.4, 442.6, 1.75, 79.6, 1.18, 98.6, 0.83, 4.5, 1.29, '良好', '輕度緊繃', '中度', '正常範圍', '正常範圍', '正常範圍', '正常範圍', '正常範圍', '正常範圍', '輕度', '正常範圍', '88 分 (優) 【建議每二個月量測一次】', '【未分類B】'], ('024', '2024/06/07 09:57'): ['024', 'YA-LIN', '女性', '1980/02/03', 44, '優於同年齡', '2024/06/07 09:57', '坐姿', 61, 18.1, 28.8, 1.15, 374.3, 155.8, 157.2, 0.98, 68.0, 1.18, 61.3, 0.5, 2.6, 1.2, '良好', '輕度緊繃', '中度', '輕度', '正常範圍', '正常範圍', '輕度', '正常範圍', '正常範圍', '中度', '正常範圍', '86 分 (優) 【建議每二個月量測一次】', '【類型一】'], ('021', '2024/06/07 09:50'): ['021', 'Haoyi', '男性', '1985/08/30', 38, '優於同年齡', '2024/06/0

NameError: name 'plot_skin_pixel_changes' is not defined

In [ ]:
# 绘制膚色变化曲线
def plot_skin_pixel_changes(skin_pixels_changes, face_positions, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    for ID, changes in skin_pixels_changes.items():
        plt.figure(figsize=(10, 5))
        for position in face_positions:
            frames = [int(frame) for _, frame, _ in changes[position]]
            changes_list = [change for _, _, change in changes[position]]
            plt.plot(frames, changes_list, label=position)
        
        plt.xlabel('Frame')
        plt.ylabel('Skin Pixels Change')
        plt.title(f'Skin Pixels Change over Frames for ID {ID}')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'{output_folder}/skin_pixels_change_{ID}.png')
        plt.close()

output_folder = 'HSV_output'
plot_skin_pixel_changes(skin_pixels_changes, face_positions, output_folder)
print(f"Skin pixel change plots have been saved to {output_folder}")

Skin pixel change plots have been saved to HSV_output


In [ ]:
# 绘制膚色变化曲线
output_folder = 'HSV_output'
plot_skin_pixel_changes(skin_pixels_changes, face_positions, output_folder)
print(f"Skin pixel change plots have been saved to {output_folder}")